End to end machine learning workflow to predict mortality within 180 days of last visit


This is step 6


The goal is this file is create test set

# Import Packages and Data

In [1]:
# Package Import

import pandas as pd

In [2]:
# import data

person = pd.read_csv('input/test_person.csv')
death = pd.read_csv('input/test_death.csv')

In [3]:
# selecting the person_id yob gender 
# reset index 

person = person[['person_id','year_of_birth']]
person = person.reset_index(drop=True)

In [4]:
# select person_id and death_datetime as the features we need 
# reset index

death = death[['person_id','death_datetime']]
death = death.reset_index(drop=True)

In [5]:
# We create a merged table left joining person and death on person_id
# reset index

demographics = person.merge(death,how='left',on='person_id')
demographics = demographics.reset_index(drop=True)

In [6]:
# Exploratory analysis
# we check how many patients have NA values for year_of_birth

len(demographics.loc[pd.isna(demographics.year_of_birth)])

4

In [7]:
# Exploratory analysis
# we check how many patients have NA values for death_date 
# Assumption: They are alive)

len(demographics.loc[pd.isna(demographics.death_datetime)])

28169

In [8]:
# Exploratory analysis
# we check how many patients have died

len(demographics.loc[pd.notna(demographics.death_datetime)])

319

In [9]:
#Preview demographics 

demographics.head()

,person_id,year_of_birth,death_datetime
0,9879,1921.0,NaN
1,640,1919.0,NaN
2,35370,1942.0,NaN
3,11594,1934.0,NaN
4,24438,1934.0,NaN


# Visits

In [16]:
from datetime import datetime

visit = pd.read_csv('input/test_visit_occurrence.csv')

In [17]:
# Selecting person_id and visit_start_datetime as the features we need from visit
# reset index

visit = visit[['person_id','visit_start_datetime']]
visit = visit.reset_index(drop=True)

In [18]:
# lets find the latest visit date for each person


# first we create a list of the unique person id's 
person_id_list = list(demographics['person_id'])



# For each person we will create a filtered data frame from visits to only include data from that patient
# Then we will select the latest date and store that into a list
latest_visit_date = []
for person in person_id_list:
    filtered_df = visit.loc[visit.person_id == person]
    filtered_df = filtered_df.reset_index(drop=True)
    latest_visit_date.append(filtered_df['visit_start_datetime'].max())
    


# we then add the latest visit date to the demographics table
demographics['latest_visit_date'] = latest_visit_date

In [19]:
# we remove patients with NA value for latest_visit_date

demographics = demographics.loc[pd.notna(demographics['latest_visit_date'])]
demographics = demographics.reset_index(drop=True)

In [20]:
# Let's create the a new demographic feature, the age of the patient during their last visit
# we convert the string datetime to an actual date time object 
# we selct the year value and convert to an integer
# we subtract the birth year 

age_last_visit = []
for i in range(len(demographics)):
    age_last_visit.append(int(datetime.strptime(demographics.loc[i]['latest_visit_date'], '%Y-%m-%d %H:%M:%S').year) - demographics.loc[i]['year_of_birth'])
    
    
demographics['age_last_visit'] = age_last_visit

In [21]:
# lets create the outcome feature
# if there is no deathdate or the date difference of death date - latest visit > 180 days outcome = 0
# else outcome = 1
# 1 is died within 180 days of last visit
# 0 is did not die within 180 days of last visit

outcome = []
for i in range(len(demographics)):
    if(pd.isna(demographics.loc[i]['death_datetime'])==True):
        outcome.append(0)
    elif((datetime.strptime(demographics.loc[i]['death_datetime'], '%Y-%m-%d %H:%M:%S') - datetime.strptime(demographics.loc[i]['latest_visit_date'], '%Y-%m-%d %H:%M:%S')).days <= 180):                       
        outcome.append(1)
    else:
        outcome.append(0)

demographics['outcome'] = outcome    
    

In [22]:
# lets count how many of each class we have

print("Number of class 1: ",len(demographics.loc[demographics.outcome==1]))
print("Number of class 0: ",len(demographics.loc[demographics.outcome==0]))

Number of class 1:  130
Number of class 0:  28354


# Charlson Index

In [23]:
conditions = pd.read_csv('input/test_condition_occurrence.csv')

In [24]:
# Filter conditions to only include the features we need
# reset index

conditions = conditions[['person_id','condition_concept_id']]
conditions = conditions.reset_index(drop=True)

In [25]:
# Create a list of the person id's from out training set

patients = list(demographics['person_id'])

In [26]:
# Store Charlson index concept id's and the weight

condition_concept_id = [312337.0,319835.0,321052.0,441002.0,4182210.0,439276.0,4043378.0,
                        374888.0,375791.0,80809.0,441928.0,257628.0,80800.0,192680.0,200763.0,
                        201254.0,201826.0,374022.0,443611.0,443597.0,443612.0,317510.0,432571.0,433740.0]

concept_weight = [1,1,1,1,1,1,1,1,1,1,1,1,1,3,1,1,1,2,2,2,2,2,2,1]

In [27]:
# filter the conditions table to only include the concept id's we need and the patients id from training set

conditions = conditions.loc[conditions.condition_concept_id.isin(condition_concept_id)]
conditions = conditions.reset_index(drop=True)

conditions = conditions.loc[conditions.person_id.isin(patients)]
conditions = conditions.reset_index(drop=True)

In [28]:
# Create a list containing a string version of each concept id to use as column names

condition_concept_id_string = []
for i in condition_concept_id:
    condition_concept_id_string.append(str(i))

In [29]:
# Create a new data frame and initialize each value to 0
# The rows will be unique patients
# the columns will be the concept codes from the charlson index


num_person = len(patients) #rows
num_condition_concept_id = len(condition_concept_id) #columns


populated_data = []
for i in range(len(patients)):
    current_row = []
    for j in range(num_condition_concept_id):
        current_row.append(0)
    populated_data.append(current_row)
    
    
    
# Convert the 2 dimensional list to a pandas df
# Set the columns to be the list of string concept id's
populated_data = pd.DataFrame(populated_data,columns=condition_concept_id_string)

# Insert the patient id's as the first column so we can reference each patient later
populated_data.insert(loc=0, column='person_id', value=patients)

In [30]:
# We Populate our df by going through each row in conditions 
# Conditions has two columns: person_id and condition_concept_id
# We store the current person and the current concept_id
# We then loop through the patients list and find the index of the person we want
# Then we just reference the df location by matching the person index and string of concept id


for i in range(len(conditions)):
    person = conditions.loc[i][0]
    concept_id = conditions.loc[i][1]

    for j in range(len(patients)):
        if(patients[j] == person):
            person_index = j
    populated_data.loc[person_index][str(concept_id)] = 1

In [32]:
# Since several Charlson Index features have multiple concept codes we need to go through and check
# each concept code and populate either the weight, or if it has multple, select the max value
# from each category. For example: a patient can have several dimensia codes so the max will just be 1
# we then store that into a list 


weighted_charlson = []
for i in range(len(populated_data)):
    s = 0 
    #Mi
    if(populated_data.loc[i]['312337.0']==1):
        s = s+1
    #CHF
    if(populated_data.loc[i]['319835.0']==1):
        s = s+1
    # PVD
    if(populated_data.loc[i]['321052.0']==1):
        s = s+1
    #Dementia
    s = s + max(populated_data.loc[i]['441002.0'],populated_data.loc[i]['4182210.0'],populated_data.loc[i]['439276.0'],populated_data.loc[i]['4043378.0'],populated_data.loc[i]['374888.0'],populated_data.loc[i]['375791.0'])
    #Connective tissue disease
    s = s + max(populated_data.loc[i]['80809.0'],populated_data.loc[i]['441928.0'],populated_data.loc[i]['257628.0'],populated_data.loc[i]['80800.0'])
    # Liver disease
    if(populated_data.loc[i]['192680.0']==1):
        s = s+3
    if(populated_data.loc[i]['200763.0']==1):
        s = s+1
    #Diabetes Mellitus
    if(populated_data.loc[i]['201254.0']==1):
        s = s+1
    if(populated_data.loc[i]['201826.0']==1):
        s = s+1
    #Hemiplegia
    if(populated_data.loc[i]['374022.0']==1):
        s = s+2
    # Chronic kidney disease 3-5
    s = s + 2*max(populated_data.loc[i]['443611.0'],populated_data.loc[i]['443597.0'],populated_data.loc[i]['443612.0'])
    # Leukemia
    if(populated_data.loc[i]['317510.0']==1):
        s = s+2
    #Lymphoma
    if(populated_data.loc[i]['432571.0']==1):
        s = s+2


    weighted_charlson.append(s)
  


In [33]:
# Lets add our new weighted charlson feature to the populated data frame
# Then we filter the df to only include the person_id and charlson score

populated_data['weighted_charlson']=weighted_charlson
populated_data = populated_data.reset_index(drop=True)
populated_data = populated_data[['person_id','weighted_charlson']]
populated_data = populated_data.reset_index(drop=True)

In [34]:
# We join the populated_data to our demographics so we can adjust charlson score by age

demographics = demographics.join(populated_data.set_index('person_id'), on='person_id')

In [35]:
demographics

,person_id,year_of_birth,death_datetime,latest_visit_date,age_last_visit,outcome,weighted_charlson
0,9879,1921.0,NaN,2009-03-09 08:32:39,88.0,0,0
1,640,1919.0,NaN,2008-06-15 09:53:33,89.0,0,0
2,35370,1942.0,NaN,2009-10-21 08:48:08,67.0,0,1
3,11594,1934.0,NaN,2009-06-23 01:18:28,75.0,0,5
4,24438,1934.0,NaN,2010-05-12 00:28:09,76.0,0,0
...,...,...,...,...,...,...,...
28479,38545,1974.0,NaN,2010-05-28 11:16:26,36.0,0,1
28480,2286,1948.0,NaN,2010-05-03 08:27:15,62.0,0,0
28481,42272,1925.0,NaN,2010-05-21 06:51:42,85.0,0,2
28482,16967,1939.0,NaN,2010-05-31 05:53:37,71.0,0,0


In [36]:
# Let's update Charlson index to include age
# We add the age score from the charlson index paper to the current score

updated = []
for i in range(len(demographics)):
    age_score = 0
    if(demographics.loc[i]['age_last_visit'] in range(50,60)):
        age_score = 1
    elif(demographics.loc[i]['age_last_visit'] in range(60,70)):
        age_score = 2
    elif(demographics.loc[i]['age_last_visit'] in range(70,80)):
        age_score = 3
    elif(demographics.loc[i]['age_last_visit'] >= 80):
        age_score = 4
  
    updated_score = age_score + demographics.loc[i]['weighted_charlson']
  
    updated.append(updated_score)

demographics['age_Charlson_index'] = updated

In [37]:
# Lets explore the charlson index breakdown between two outcomes

demographics[['age_Charlson_index','outcome']].groupby('outcome').mean()

,age_Charlson_index
outcome,
0,4.512309
1,4.792308


In [38]:
# Lets filter our demographics to only include the features we need
# reset index

demographics = demographics[['person_id','outcome','age_Charlson_index','age_last_visit']]
demographics = demographics.reset_index(drop=True)




# Conditions

In [57]:
conditions = pd.read_csv('input/test_condition_occurrence.csv')

In [58]:
# Select the features we need from conditions

conditions = conditions[['person_id','condition_concept_id']]
conditions = conditions.reset_index(drop=True)

In [59]:
# create a list of each patient

patients = list(demographics['person_id'])

In [60]:
# filter conditions to only include conditions from the patients we have

conditions = conditions.loc[conditions.person_id.isin(patients)]
conditions = conditions.reset_index(drop=True)

In [61]:
testing_features = [313217.0,439926.0,75617.0,4229909.0,317576.0,444369.0]

conditions = conditions.loc[conditions.condition_concept_id.isin(testing_features)]
conditions = conditions.reset_index(drop=True)





In [62]:
condition_names = list(set(conditions['condition_concept_id']))

In [63]:
# Transform the condition names into strings

storage_list = []
for i in range(len(condition_names)):
    storage_list.append(str(condition_names[i]))
    
conditions_names = storage_list

In [64]:
# Populate a df with initialized values as 0

columns = len(conditions_names)
rows = len(patients)


populated_conditions = []

for i in range(rows):
    current_row = []
    for j in range(columns):
        current_row.append(0)
    populated_conditions.append(current_row)
    
populated_conditions = pd.DataFrame(populated_conditions,columns = conditions_names)

populated_conditions.insert(loc=0, column='person_id', value=patients)

In [65]:
# We Populate our df by going through each row in conditions 
# Conditions has two columns: person_id and condition_concept_id
# We store the current person and the current concept_id
# We then loop through the patients list and find the index of the person we want
# Then we just reference the df location by matching the person index and string of concept id


for i in range(len(conditions)):
    current_person = conditions.loc[i][0]
    concept_id = conditions.loc[i][1]

    for j in range(len(patients)):
        if(patients[j] == current_person):
            person_index = j

    populated_conditions.loc[person_index][str(concept_id)] = 1

In [66]:
populated_conditions

,person_id,313217.0,75617.0,317576.0,444369.0,4229909.0,439926.0
0,9879,0,0,0,0,0,0
1,640,0,0,0,0,0,0
2,35370,0,0,0,0,0,0
3,11594,1,1,1,0,0,1
4,24438,0,0,1,0,0,0
...,...,...,...,...,...,...,...
28479,38545,1,0,0,0,0,1
28480,2286,0,0,0,0,0,0
28481,42272,1,0,1,0,0,1
28482,16967,0,0,0,0,0,0


In [67]:
# Join to demographics

demographics = demographics.join(populated_conditions.set_index('person_id'), on='person_id')
demographics = demographics.reset_index(drop=True)

In [68]:
demographics

,person_id,outcome,age_Charlson_index,age_last_visit,313217.0,75617.0,317576.0,444369.0,4229909.0,439926.0
0,9879,0,4,88.0,0,0,0,0,0,0
1,640,0,4,89.0,0,0,0,0,0,0
2,35370,0,3,67.0,0,0,0,0,0,0
3,11594,0,8,75.0,1,1,1,0,0,1
4,24438,0,3,76.0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
28479,38545,0,1,36.0,1,0,0,0,0,1
28480,2286,0,2,62.0,0,0,0,0,0,0
28481,42272,0,6,85.0,1,0,1,0,0,1
28482,16967,0,3,71.0,0,0,0,0,0,0


In [69]:
demographics.to_csv('output/test_set.csv',index=False)